# Calculate average size of OWF from Turbine number and engineering constraints

*Calculating approximate boundary of wind farms given no. of turbines*
1. Each turbine requires 500m spacing from surrounding turbines/
2. Assuming grid layout, number of turbines on a is sqrt(num_turbines)
3. Translating lat/lon of centre of known site to OS grid Easting and Northing coordinates, coordinates can be mapped to a 10x10m national grid
4. Positions of site boundaries can be calculated on this grid, then translated back into lat/lon coordinates
5. The offsets calculated from this can then be used to workout average size of farms, giving useful limitiations for finding new sites

In [7]:
import pandas as pd
import os
from OSGridConverter import latlong2grid, grid2latlong
import numpy as np

In [2]:
os.chdir('C:/Users/zacha/Documents/GitHub/plymouth-university-proj518/data')
print(os.getcwd())

C:\Users\zacha\Documents\GitHub\plymouth-university-proj518\data


In [3]:
file = 'UK east coast wind energy locations.csv'
df = pd.read_csv(file, sep=',')

In [8]:
lon_avg = 0
lat_avg = 0

for idx, row in df.iterrows():
    lon, lat, turbnum = row['LNG'], row['LAT'], row['TURBNUM']

    grid = latlong2grid(lat, lon)
    easting, northing = grid.eastNorth()

    # square root turbnum to get number of turbines along lon / lat dimensions
    length_turbnum = np.sqrt(turbnum)

    # length of lon / lat dimensions from 500m radius / turbine in metres
    # get offset from 500m / 2.0 for radius, then divide by 10.0 to get northing/eassting conversion to 10x10m national grid = 25.0
    eastNorth_offset = int(length_turbnum * 25.0)

    # calculate easting northings of corners
    q1_easting, q1_northing = easting - eastNorth_offset, northing + eastNorth_offset
    q2_easting, q2_northing = easting + eastNorth_offset, northing + eastNorth_offset
    q3_easting, q3_northing = easting - eastNorth_offset, northing - eastNorth_offset
    q4_easting, q4_northing = easting + eastNorth_offset, northing - eastNorth_offset

    # calculate lons and lats of corners
    q1 = grid2latlong(f'TB {str(q1_easting)[1:]} {str(q1_northing)[1:]}', tag='WGS84')
    q2 = grid2latlong(f'TB {str(q2_easting)[1:]} {str(q2_northing)[1:]}', tag='WGS84')
    q3 = grid2latlong(f'TB {str(q3_easting)[1:]} {str(q3_northing)[1:]}', tag='WGS84')
    q4 = grid2latlong(f'TB {str(q4_easting)[1:]} {str(q4_northing)[1:]}', tag='WGS84')

    q1_lon, q1_lat = q1.longitude, q1.latitude
    q2_lon, q2_lat = q2.longitude, q2.latitude
    q3_lon, q3_lat = q3.longitude, q3.latitude
    q4_lon, q4_lat = q4.longitude, q4.latitude

    # calculate offsets from centre
    q1_lon_offset = q1_lon - lon
    q1_lat_offset = q1_lat - lat
    q2_lon_offset = q2_lon - lon
    q2_lat_offset = q2_lat - lat
    q3_lon_offset = q3_lon - lon
    q3_lat_offset = q3_lat - lat
    q4_lon_offset = q4_lon - lon
    q4_lat_offset = q4_lat - lat

    # calculate average lon / lat
    lon_avg += q1_lon_offset + q2_lon_offset + q3_lon_offset + q4_lon_offset
    lat_avg += q1_lat_offset + q2_lat_offset + q3_lat_offset + q4_lat_offset

    lon_avg /= 4
    lat_avg /= 4



lon_avg /= len(df)
lat_avg /= len(df)

print(f"Lon avg: {lon_avg}, Lat avg: {lat_avg}, from {len(df)} sites")

Lon avg: 0.0010898192983598688, Lat avg: -2.1518227154293523e-07, from 4 sites
